# 🎤 Codename RVC Fork 4 - Modo CLI

##### Notebook made by shiroug :shiba:

In [ ]:
# @title 1️⃣ Clonar Repositório e Configurar Ambiente

import os
import sys

# === CLONAR O REPOSITÓRIO ===
print("📥 Clonando repositório...")
!git clone https://github.com/ShiromiyaG/codename-rvc-fork-4.git
%cd codename-rvc-fork-4

# === PREPARAR PYTHON 3.10 ===
print("\n🔄 Configurando Python 3.10...")
!apt-get update -qq
!apt-get install -y -qq python3.10 python3.10-dev python3.10-distutils libpython3.10-dev curl aria2

# Atualizar alternativas para usar Python 3.10
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!update-alternatives --set python3 /usr/bin/python3.10

# === INSTALAR UV ===
print("\n📦 Instalando gerenciador de pacotes UV...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ["PATH"] += ":/root/.cargo/bin"

# Instalar pip e dependências
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
!uv pip install --system -r requirements.txt

print("\n✅ Configuração concluída!")
!python3 --version

In [ ]:
# @title 2️⃣ Baixar Pretrains Customizados (Hugging Face)

# @markdown ### Gerador (G)
G_url = "" # @param {type:"string"}
G_filename = "G_custom.pth" # @param {type:"string"}

# @markdown ### Discriminador (D)
D_url = "" # @param {type:"string"}
D_filename = "D_custom.pth" # @param {type:"string"}

import subprocess
from pathlib import Path

PRETRAIN_DIR = Path("rvc/models/pretraineds/custom")

def download_file(url, output_dir, filename):
    """Baixa um arquivo focado em links do Hugging Face"""
    if not url.strip():
        return
    
    # Lógica Específica para Hugging Face
    if "huggingface.co" in url:
        # Troca 'blob' (visualização) por 'resolve' (download direto)
        url = url.replace("/blob/", "/resolve/")
        
        # Remove parâmetros de query desnecessários se existirem (ex: ?download=true)
        if "?" in url:
            url = url.split("?")[0]
            
    # Cria o diretório se não existir
    output_dir.mkdir(parents=True, exist_ok=True)
    full_path = output_dir / filename
    
    if full_path.exists():
        print(f"⏭️ {filename} já existe")
        return
    
    print(f"📥 Baixando {filename} de: {url}")
    
    # Executa o download usando aria2c
    # -x 16 -s 16: Múltiplas conexões para velocidade máxima
    try:
        subprocess.run(
            ['aria2c', '-x', '16', '-s', '16', '-d', str(output_dir), '-o', filename, url],
            check=True
        )
        if full_path.exists():
            print(f"✅ {filename} baixado com sucesso!")
        else:
            print(f"❌ Falha ao baixar {filename}. Verifique o link.")
    except subprocess.CalledProcessError:
        print(f"❌ Erro na execução do download para {filename}")

# Execução
if G_url:
    download_file(G_url, PRETRAIN_DIR, G_filename)
if D_url:
    download_file(D_url, PRETRAIN_DIR, D_filename)

---
# 🎓 FLUXO DE TREINO

O pipeline de treino consiste em 3 etapas:
1. **Preprocess** → Preparar e limpar arquivos de áudio
2. **Extract** → Extrair pitch (F0) e embeddings de voz  
3. **Train** → Treinar o modelo

---

In [ ]:
# @title 4️⃣ Etapa 1: Preprocessar Dataset
# @markdown Prepara os arquivos de áudio para treino (corte, normalização, etc.)
# @markdown 
# @markdown > ⚠️ **Nota:** Se `CUT_MODE = "Simple"`, os áudios terão o silêncio truncado automaticamente antes do preprocess.

# @markdown ### Configurações Obrigatórias
MODEL_NAME = "meu_modelo_de_voz"  # @param {type:"string"}
DATASET_PATH = "assets/datasets/meu_dataset"  # @param {type:"string"}
SAMPLE_RATE = 32000  # @param [24000, 32000, 40000, 48000] {type:"raw"}

# @markdown ### Configurações Opcionais
CUT_MODE = "Simple"  # @param ["Skip", "Simple", "Automatic"]
NORMALIZATION = "post_lufs"  # @param ["none", "post_lufs", "post_peak"]
CHUNK_LEN = 3.0  # @param {type:"slider", min:0.5, max:5.0, step:0.5}

# @markdown ### Configurações de Truncar Silêncio (apenas para Simple)
# @markdown Baseado no "Travar Silêncio" do Audacity
SILENCE_THRESHOLD_DB = -50  # @param {type:"slider", min:-70, max:-20, step:5}
MIN_SILENCE_DURATION = 0.5  # @param {type:"slider", min:0.1, max:2.0, step:0.1}
KEEP_SILENCE = 0.3  # @param {type:"slider", min:0.0, max:1.0, step:0.1}

import os
import shutil
from pathlib import Path
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

def truncate_silence(audio_path, output_path, threshold_db=-50, min_silence_ms=500, keep_silence_ms=300):
    """
    Remove silêncio do início e fim do áudio.
    Baseado nas configurações do "Travar Silêncio" do Audacity.
    
    Args:
        audio_path: Caminho do áudio original
        output_path: Caminho para salvar o áudio processado
        threshold_db: Limiar em dB para detectar silêncio (padrão: -50dB)
        min_silence_ms: Duração mínima de silêncio em ms (padrão: 500ms)
        keep_silence_ms: Quanto silêncio manter nas bordas em ms (padrão: 300ms)
    """
    try:
        # Carregar áudio
        audio = AudioSegment.from_file(audio_path)
        
        # Detectar partes não-silenciosas
        nonsilent_ranges = detect_nonsilent(
            audio,
            min_silence_len=min_silence_ms,
            silence_thresh=threshold_db
        )
        
        if not nonsilent_ranges:
            # Áudio é todo silêncio, pular
            return False
        
        # Pegar início da primeira parte não-silenciosa e fim da última
        start_ms = max(0, nonsilent_ranges[0][0] - keep_silence_ms)
        end_ms = min(len(audio), nonsilent_ranges[-1][1] + keep_silence_ms)
        
        # Cortar o áudio
        trimmed_audio = audio[start_ms:end_ms]
        
        # Salvar
        output_path.parent.mkdir(parents=True, exist_ok=True)
        trimmed_audio.export(output_path, format=output_path.suffix[1:])
        
        return True
    except Exception as e:
        print(f"   ⚠️ Erro ao processar {audio_path.name}: {e}")
        return False

def process_dataset_silence(dataset_path, threshold_db, min_silence_sec, keep_silence_sec):
    """
    Processa todos os áudios do dataset, truncando silêncio.
    Cria uma cópia temporária do dataset com os áudios processados.
    """
    dataset_path = Path(dataset_path)
    temp_dataset = Path(f"/tmp/dataset_trimmed_{dataset_path.name}")
    
    # Limpar pasta temporária se existir
    if temp_dataset.exists():
        shutil.rmtree(temp_dataset)
    temp_dataset.mkdir(parents=True, exist_ok=True)
    
    # Encontrar todos os arquivos de áudio
    audio_extensions = {'.wav', '.mp3', '.flac', '.ogg', '.m4a', '.opus'}
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(dataset_path.rglob(f"*{ext}"))
    
    if not audio_files:
        print("⚠️ Nenhum arquivo de áudio encontrado no dataset!")
        return None
    
    print(f"🔇 Truncando silêncio de {len(audio_files)} arquivos...")
    print(f"   Limiar: {threshold_db}dB | Duração mín: {min_silence_sec}s | Manter: {keep_silence_sec}s")
    print("="*60)
    
    min_silence_ms = int(min_silence_sec * 1000)
    keep_silence_ms = int(keep_silence_sec * 1000)
    
    processed = 0
    skipped = 0
    
    for audio_file in audio_files:
        # Manter estrutura de pastas relativa
        relative_path = audio_file.relative_to(dataset_path)
        output_path = temp_dataset / relative_path
        
        if truncate_silence(audio_file, output_path, threshold_db, min_silence_ms, keep_silence_ms):
            processed += 1
        else:
            skipped += 1
        
        # Mostrar progresso a cada 10 arquivos
        if (processed + skipped) % 10 == 0:
            print(f"   Processados: {processed + skipped}/{len(audio_files)}")
    
    print(f"\n✅ Silêncio truncado: {processed} arquivos processados, {skipped} ignorados")
    return str(temp_dataset)

# === EXECUTAR PREPROCESSAMENTO ===
print(f"📂 Modelo: {MODEL_NAME}")
print(f"📂 Dataset: {DATASET_PATH}")
print(f"🎵 Taxa de Amostragem: {SAMPLE_RATE}Hz")
print(f"✂️ Modo de Corte: {CUT_MODE}")
print("="*60)

# Se o modo for Simple, truncar silêncio primeiro
final_dataset_path = DATASET_PATH

if CUT_MODE == "Simple":
    print("\n🔇 Modo Simple detectado - Truncando silêncio do dataset...\n")
    processed_path = process_dataset_silence(
        DATASET_PATH,
        threshold_db=SILENCE_THRESHOLD_DB,
        min_silence_sec=MIN_SILENCE_DURATION,
        keep_silence_sec=KEEP_SILENCE
    )
    if processed_path:
        final_dataset_path = processed_path
        print(f"\n📂 Usando dataset processado: {final_dataset_path}\n")
    print("="*60)

print("\n🚀 Iniciando preprocessamento CLI...\n")

!python core.py preprocess \
    --model_name "{MODEL_NAME}" \
    --dataset_path "{final_dataset_path}" \
    --sample_rate {SAMPLE_RATE} \
    --cut_preprocess "{CUT_MODE}" \
    --normalization_mode "{NORMALIZATION}" \
    --chunk_len {CHUNK_LEN} \
    --loading_resampling "ffmpeg" \
    --lufs_range_finder True

print("\n✅ Preprocessamento concluído!")

In [ ]:
# @title 5️⃣ Etapa 2: Extrair Features
# @markdown Extrai pitch (F0) e embeddings de voz dos áudios preprocessados

# @markdown ### Configurações
F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
EMBEDDER = "contentvec"  # @param ["contentvec", "spin_v1", "spin_v2", "custom"]
VOCODER_ARCH = "hifi_refine"  # @param ["hifi_refine", "ringformer", "pcph_gan"]

print(f"📂 Modelo: {MODEL_NAME}")
print(f"🎤 Método F0: {F0_METHOD}")
print(f"🔊 Embedder: {EMBEDDER}")
print("="*60)

!python core.py extract \
    --model_name "{MODEL_NAME}" \
    --sample_rate {SAMPLE_RATE} \
    --f0_method "{F0_METHOD}" \
    --embedder_model "{EMBEDDER}" \
    --vocoder_arch "{VOCODER_ARCH}" \
    --include_mutes 2 \
    --gpu "0"

print("\n✅ Extração de features concluída!")

In [ ]:
# @title 6️⃣ Etapa 3: Treinar Modelo
# @markdown Treina o modelo RVC usando as features extraídas

# @markdown ### Configurações Obrigatórias
TOTAL_EPOCHS = 300  # @param {type:"integer"}
BATCH_SIZE = 4  # @param {type:"slider", min:1, max:32, step:1}

# @markdown ### Arquitetura
VOCODER = "HiFi-GAN"  # @param ["HiFi-GAN", "PCPH-GAN", "RefineGAN", "RingFormer_v1", "RingFormer_v2"]
ARCHITECTURE = "RVC"  # @param ["RVC", "Fork/Applio", "Fork"]
OPTIMIZER = "AdamW"  # @param ["AdamW BF16", "AdamW", "AdamSPD", "RAdam", "Ranger21", "DiffGrad", "Prodigy"]

# @markdown ### Modelo Pré-treinado
USE_PRETRAINED = True  # @param {type:"boolean"}
CUSTOM_PRETRAINED = False  # @param {type:"boolean"}
G_PATH = "rvc/models/pretraineds/custom/G_custom.pth"  # @param {type:"string"}
D_PATH = "rvc/models/pretraineds/custom/D_custom.pth"  # @param {type:"string"}

# @markdown ### Configurações de Salvamento
SAVE_EVERY = 10  # @param {type:"integer"}
SAVE_ONLY_LATEST = True  # @param {type:"boolean"}

# @markdown ### Avançado
USE_WARMUP = False  # @param {type:"boolean"}
WARMUP_EPOCHS = 10  # @param {type:"integer"}
SPECTRAL_LOSS = "L1 Mel Loss"  # @param ["L1 Mel Loss", "Multi-Scale Mel Loss", "Multi-Res STFT Loss"]
LR_SCHEDULER = "exp decay step"  # @param ["exp decay step", "exp decay epoch", "cosine annealing", "none"]
VITS_2_MODE = False # @param {type:"boolean"}
CLEANUP = True # @param {type:"boolean"}


print(f"📂 Modelo: {MODEL_NAME}")
print(f"🎵 Taxa de Amostragem: {SAMPLE_RATE}Hz")
print(f"🔢 Total de Epochs: {TOTAL_EPOCHS}")
print(f"📦 Tamanho do Batch: {BATCH_SIZE}")
print(f"🎛️ Vocoder: {VOCODER}")
print("="*60)

# Montar comando com argumentos opcionais
cmd = f"""python core.py train \\
    --model_name "{MODEL_NAME}" \\
    --sample_rate {SAMPLE_RATE} \\
    --total_epoch_count {TOTAL_EPOCHS} \\
    --batch_size {BATCH_SIZE} \\
    --vocoder "{VOCODER}" \\
    --architecture "{ARCHITECTURE}" \\
    --optimizer "{OPTIMIZER}" \\
    --epoch_save_frequency {SAVE_EVERY} \\
    --save_only_latest_net_models {str(SAVE_ONLY_LATEST)} \\
    --save_weight_models True \\
    --pretrained {str(USE_PRETRAINED)} \\
    --custom_pretrained {str(CUSTOM_PRETRAINED)} \\
    --use_warmup {str(USE_WARMUP)} \\
    --warmup_duration {WARMUP_EPOCHS} \\
    --spectral_loss "{SPECTRAL_LOSS}" \\
    --lr_scheduler "{LR_SCHEDULER}" \\
    --use_validation False \\
    --vits_2_mode {str(VITS_2_MODE)} \\
    --cleanup {str(CLEANUP)} \\
    --gpu "0"""

# Adicionar caminhos de pretrain se usando custom
if CUSTOM_PRETRAINED and G_PATH:
    cmd += f' --g_pretrained_path "{G_PATH}"'
if CUSTOM_PRETRAINED and D_PATH:
    cmd += f' --d_pretrained_path "{D_PATH}"'

print("\n🚀 Iniciando treino...")
print("💡 Dica: Use Ctrl+C para parar o treino a qualquer momento\n")
!{cmd}

In [ ]:
# @title 7️⃣ Gerar Arquivo Index
# @markdown Cria um arquivo de índice para o modelo treinado (usado na inferência)

INDEX_ALGORITHM = "Auto"  # @param ["Auto", "Faiss", "KMeans"]

print(f"📂 Modelo: {MODEL_NAME}")
print("="*60)

!python core.py index \
    --model_name "{MODEL_NAME}" \
    --index_algorithm "{INDEX_ALGORITHM}"

print("\n✅ Arquivo index gerado!")

---
# 🎙️ INFERÊNCIA (Conversão de Voz)

Converta áudios usando seu modelo treinado.

---

In [ ]:
# @title 8️⃣ Inferência de Arquivo Único
# @markdown Converte um único arquivo de áudio

# @markdown ### Caminhos Obrigatórios
INPUT_PATH = "/kaggle/input/audio/musica.wav"  # @param {type:"string"}
OUTPUT_PATH = "/kaggle/working/output.wav"  # @param {type:"string"}
MODEL_PATH = "logs/meu_modelo_de_voz/meu_modelo_de_voz.pth"  # @param {type:"string"}
INDEX_PATH = "logs/meu_modelo_de_voz/added_*.index"  # @param {type:"string"}

# @markdown ### Configurações de Inferência
PITCH = 0  # @param {type:"slider", min:-24, max:24, step:1}
F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
INDEX_RATE = 0.5  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
PROTECT = 0.33  # @param {type:"slider", min:0.0, max:0.5, step:0.05}

# @markdown ### Configurações de Áudio
SPLIT_AUDIO = False  # @param {type:"boolean"}
CLEAN_AUDIO = False  # @param {type:"boolean"}
F0_AUTOTUNE = False  # @param {type:"boolean"}
EXPORT_FORMAT = "WAV"  # @param ["WAV", "MP3", "FLAC", "OGG", "M4A"]

print(f"🎵 Entrada: {INPUT_PATH}")
print(f"💾 Saída: {OUTPUT_PATH}")
print(f"🎤 Modelo: {MODEL_PATH}")
print(f"🎛️ Pitch: {PITCH:+d} semitons")
print("="*60)

!python core.py infer \
    --input_path "{INPUT_PATH}" \
    --output_path "{OUTPUT_PATH}" \
    --pth_path "{MODEL_PATH}" \
    --index_path "{INDEX_PATH}" \
    --pitch {PITCH} \
    --f0_method "{F0_METHOD}" \
    --index_rate {INDEX_RATE} \
    --protect {PROTECT} \
    --split_audio {str(SPLIT_AUDIO)} \
    --clean_audio {str(CLEAN_AUDIO)} \
    --f0_autotune {str(F0_AUTOTUNE)} \
    --export_format "{EXPORT_FORMAT}"

print(f"\n✅ Saída salva em: {OUTPUT_PATH}")

In [ ]:
# @title 9️⃣ Inferência em Lote
# @markdown Converte múltiplos arquivos de áudio de uma pasta

# @markdown ### Caminhos Obrigatórios
INPUT_FOLDER = "/kaggle/input/musicas/"  # @param {type:"string"}
OUTPUT_FOLDER = "/kaggle/working/convertidos/"  # @param {type:"string"}
MODEL_PATH = "logs/meu_modelo_de_voz/meu_modelo_de_voz.pth"  # @param {type:"string"}
INDEX_PATH = "logs/meu_modelo_de_voz/added_*.index"  # @param {type:"string"}

# @markdown ### Configurações (mesmas da inferência única)
PITCH = 0  # @param {type:"slider", min:-24, max:24, step:1}
F0_METHOD = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe"]
INDEX_RATE = 0.5  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
EXPORT_FORMAT = "WAV"  # @param ["WAV", "MP3", "FLAC", "OGG", "M4A"]

import os
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print(f"📂 Pasta de entrada: {INPUT_FOLDER}")
print(f"📂 Pasta de saída: {OUTPUT_FOLDER}")
print(f"🎤 Modelo: {MODEL_PATH}")
print(f"🎛️ Pitch: {PITCH:+d} semitons")
print("="*60)

!python core.py batch_infer \
    --input_folder "{INPUT_FOLDER}" \
    --output_folder "{OUTPUT_FOLDER}" \
    --pth_path "{MODEL_PATH}" \
    --index_path "{INDEX_PATH}" \
    --pitch {PITCH} \
    --f0_method "{F0_METHOD}" \
    --index_rate {INDEX_RATE} \
    --export_format "{EXPORT_FORMAT}"

print(f"\n✅ Todos os arquivos convertidos! Confira: {OUTPUT_FOLDER}")